# GAT
\
スカラー形式の式
$$
\pmb{h}_v^{(l+1)} = \sigma \left( \sum_{u \in \mathcal{N}(v)}{\alpha_{vu}^{(l+1)} \mathbf{W}^{(l+1)} \pmb{h}_u^{l} } \right) \\
$$
$$
\alpha_{vu}^{(l+1)} = \frac{\exp \left( \text{LeakyReLU}  \left( \pmb{a}^{(l+1)T} \lbrack \mathbf{W}_a^{(l+1)} \pmb{h}_v^{l} \Vert \mathbf{W}_a^{(l+1)} \pmb{h}_u^{l} \rbrack \right) \right) } { \sum\limits_{w \in \mathcal{N}(v)} \exp \left( \text{LeakyReLU}  \left( \pmb{a}^{(l+1)T} \lbrack \mathbf{W}_a^{(l+1)} \pmb{h}_v^{l} \Vert \mathbf{W}_a^{(l+1)} \pmb{h}_w^{l} \rbrack \right) \right) }
$$

参考文献 :\
「Graph Attention Networks」 \
https://arxiv.org/abs/1710.10903 \
佐藤竜馬: 「機械学習プロフェッショナルシリーズ グラフニューラルネットワーク」

# PyTorch ベース

佐藤竜馬: 「機械学習プロフェッショナルシリーズ グラフニューラルネットワーク」 式実装 \
現論文ではWとW_aは共通の重みを使用するが、今回は別とする。

In [ ]:
class GATlayer_a(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        att_in_channels: int,
        att_out_channels: int,
        leaky_relu_slope: float = 0.2
        ):

        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.att_in_channels = att_in_channels
        self.att_out_channels = att_out_channels

        # 学習パラメータの設定
        self.a = Parameter(torch.empty(self.att_out_channels * 2, 1))
        self.W = Linear(self.in_channels, self.out_channels)
        self.W_a = Linear(self.att_in_channels, self.att_out_channels)

        self.leakyrelu = nn.LeakyReLU(leaky_relu_slope)
        self.softmax = nn.Softmax(dim=1)

        self.reset_parameters()

    def reset_parameters(self):
        self.W.reset_parameters()
        self.W_a.reset_parameters()
        init.xavier_uniform_(self.a)


    def _get_attention_scores(self,
                              h: Tensor
                              ):

        h_tilde = self.W_a(h)

        a_weighted_h_v = torch.matmul(h_tilde, self.a[:self.att_out_channels, :])
        a_weighted_h_u = torch.matmul(h_tilde, self.a[self.att_out_channels:, :])

        e = a_weighted_h_v + a_weighted_h_u.mT

        return self.leakyrelu(e)

    def forward(self,
                h: Tensor,
                edge_index: Adj
                ):

        # 隣接行列を作る
        num_nodes = h.shape[0]
        A = torch.zeros((num_nodes, num_nodes)).to(device)
        A[edge_index[0], edge_index[1]] = 1
        # 自己ループ持ち隣接行列を作る
        I = torch.eye(num_nodes).to(device)
        A = A + I

        HW = self.W(h)

        e = self._get_attention_scores(h)
        self.e = e.detach()

        connectivity_mask = -9e16 * torch.ones_like(e)
        e = torch.where(A > 0, e, connectivity_mask)

        attention = F.softmax(e, dim=1)
        self.attention = attention

        H_new = torch.matmul(attention, HW)

        return H_new